In [1]:
from sqlalchemy import create_engine, MetaData, Table, inspect, text
import pandas as pd
from mysql.connector import connect, Error
global database
import mysql.connector
# import pandas as pd
from sqlalchemy import create_engine


In [2]:
engine = create_engine('######')


In [31]:

# Consulta SQL
query = '''
    SELECT * FROM dim_asesor
'''

# Cargar los datos en un DataFrame de pandas
df = pd.read_sql(query, con=engine)

In [32]:
df

,asesor_id,nombre,especialización,años_experiencia,correo,teléfono
0,1,Pedro Martínez,Finanzas,10,pedro.martinez@asesoria.com,600111222
1,2,Laura Sánchez,Tecnología,8,laura.sanchez@asesoria.com,600222333
2,3,David Ruiz,Legal,12,david.ruiz@asesoria.com,600333444
3,4,Sofia Díaz,Marketing,5,sofia.diaz@asesoria.com,600444555
4,5,Javier Ortega,Innovación,7,javier.ortega@asesoria.com,600555666


In [5]:
# ¿Cuáles son los ingresos mensuales por área de asesoría?
 

In [10]:
query = '''
    SELECT
    df.año,
    df.mes,
    daa.nombre AS area_asesoria,
    SUM(fsp.costo_total) AS ingresos_totales
FROM
    Fact_Servicios_Prestados fsp
JOIN
    Dim_Fecha df ON fsp.fecha_id = df.fecha_id
JOIN
    Dim_Area_Asesoria daa ON fsp.area_asesoria_id = daa.area_asesoria_id
GROUP BY
    df.año, df.mes, daa.nombre
ORDER BY
    df.año, df.mes, ingresos_totales DESC;
    
'''

# Cargar los datos en un DataFrame de pandas
df = pd.read_sql(query, con=engine)

In [103]:
df

,asesor_id,nombre,especialización,años_experiencia,correo,teléfono
0,1,Pedro Martínez,Finanzas,10,pedro.martinez@asesoria.com,600111222
1,2,Laura Sánchez,Tecnología,8,laura.sanchez@asesoria.com,600222333
2,3,David Ruiz,Legal,12,david.ruiz@asesoria.com,600333444
3,4,Sofia Díaz,Marketing,5,sofia.diaz@asesoria.com,600444555
4,5,Javier Ortega,Innovación,7,javier.ortega@asesoria.com,600555666


In [13]:
# ¿Cuáles son los asesores con más horas trabajadas en cada área de asesoría?


In [38]:
query = '''
    SELECT 
    da.nombre,
    daa.nombre as area_asesoria,
    sum(fsp.horas) as Horas_Totales
FROM 
    fact_servicios_prestados fsp
JOIN 
    dim_area_asesoria daa on fsp.area_asesoria_id=daa.area_asesoria_id
JOIN
    dim_asesor da on fsp.asesor_id=da.asesor_id
GROUP BY
    da.nombre,daa.nombre
ORDER BY 
    Horas_Totales DESC;
     '''

In [39]:
df2 = pd.read_sql(query, con=engine)

In [41]:
# ¿Cuáles son los clientes que han generado más ingresos y en qué área de asesoría?


In [67]:
query = '''
    SELECT dc.nombre,
    dc.empresa,
    dc.industria,
    sum(fsp.costo_total) as ingresos,
    daa.nombre as area_asesoria
    
FROM 
    fact_servicios_prestados fsp
JOIN
    dim_cliente dc on fsp.cliente_id=dc.cliente_id
JOIN
    dim_area_asesoria daa on fsp.area_asesoria_id=daa.area_asesoria_id
GROUP BY
    dc.nombre,dc.empresa,dc.industria,daa.nombre
ORDER BY ingresos DESC;

    
     '''

In [68]:
df3 = pd.read_sql(query, con=engine)

In [69]:
df3

,nombre,empresa,industria,ingresos,area_asesoria
0,Carlos García,García Innovaciones,Innovación,10924.52,Legal
1,Carlos García,García Innovaciones,Innovación,7678.73,Tecnología
2,Marta López,López Consultores,Marketing,7614.10,Marketing
3,Luis Fernández,Fernández Asociados,Legal,6617.88,Tecnología
4,Marta López,López Consultores,Marketing,6254.04,Finanzas
5,Ana Gómez,Gómez & Co.,Tecnología,5478.70,Tecnología
6,Carlos García,García Innovaciones,Innovación,5448.16,Finanzas
7,Juan Perez,Perez S.A.,Finanzas,5432.93,Marketing
8,Ana Gómez,Gómez & Co.,Tecnología,5123.47,Innovación
9,Juan Perez,Perez S.A.,Finanzas,4746.98,Tecnología


In [70]:
# ¿Cuál es la efectividad de los asesores (ingresos por hora) por área de asesoría?
# Queremos saber los 5 mejores asesores y a que área de asesoría hace referencia



In [87]:
query = '''
    SELECT da.nombre,
    da.años_experiencia,
    daa.nombre as area_asesoria,
    sum(fsp.costo_total)/sum(fsp.horas) as ingresos_por_hora
    
FROM fact_servicios_prestados fsp

JOIN dim_asesor da on fsp.asesor_id=da.asesor_id

JOIN dim_area_asesoria daa on fsp.area_asesoria_id=daa.area_asesoria_id

GROUP BY da.nombre,da.años_experiencia,daa.nombre

ORDER BY ingresos_por_hora DESC

LIMIT   5;
    
    

    
     '''

In [88]:
df4 = pd.read_sql(query, con=engine)

In [89]:
df4

,nombre,años_experiencia,area_asesoria,ingresos_por_hora
0,David Ruiz,12,Innovación,409.560000
1,David Ruiz,12,Finanzas,213.242500
2,Laura Sánchez,8,Tecnología,187.261379
3,Laura Sánchez,8,Innovación,146.596667
4,Sofia Díaz,5,Legal,137.302857


# ¿Qué clientes han recibido servicios de múltiples áreas de asesoría y cuáles son esas áreas?

In [94]:
query = '''
    SELECT dc.nombre as cliente,
    dc.empresa, 
    GROUP_CONCAT(DISTINCT daa.nombre ORDER BY daa.nombre) AS areas_asesoria
    
    FROM fact_servicios_prestados fsp
    
    JOIN dim_cliente dc on fsp.cliente_id=dc.cliente_id
    
    JOIN dim_area_asesoria daa on fsp.area_asesoria_id=daa.area_asesoria_id
    
    GROUP BY dc.nombre,dc.empresa
    
    HAVING COUNT(DISTINCT fsp.area_asesoria_id) > 1
    
    ORDER BY dc.nombre

     '''

In [95]:
df5 = pd.read_sql(query, con=engine)

In [96]:
df5

,cliente,empresa,areas_asesoria
0,Ana Gómez,Gómez & Co.,"Finanzas,Innovación,Legal,Marketing,Tecnología"
1,Carlos García,García Innovaciones,"Finanzas,Innovación,Legal,Marketing,Tecnología"
2,Juan Perez,Perez S.A.,"Finanzas,Innovación,Legal,Marketing,Tecnología"
3,Luis Fernández,Fernández Asociados,"Innovación,Legal,Marketing,Tecnología"
4,Marta López,López Consultores,"Finanzas,Innovación,Legal,Marketing,Tecnología"


# ¿Cuál es la tendencia de crecimiento mensual de ingresos totales?



In [98]:
query = '''
    SELECT
    df.año,
    df.mes,
    SUM(fsp.costo_total) AS ingresos_totales,
    SUM(fsp.costo_total) - LAG(SUM(fsp.costo_total)) OVER (ORDER BY df.año, df.mes) AS crecimiento_mensual
FROM
    Fact_Servicios_Prestados fsp
JOIN
    Dim_Fecha df ON fsp.fecha_id = df.fecha_id
GROUP BY
    df.año, df.mes
ORDER BY
    df.año, df.mes;


     '''

In [100]:
df6 = pd.read_sql(query, con=engine)
df6

,año,mes,ingresos_totales,crecimiento_mensual
0,2023,1,34411.06,NaN
1,2023,2,30778.56,-3632.50
2,2023,3,30496.46,-282.10
3,2023,4,11667.64,-18828.82


# ¿Cuál es la relación entre la experiencia del asesor y los ingresos generados?



In [101]:
query = '''
    SELECT
    da.nombre AS asesor,
    da.años_experiencia,
    SUM(fsp.costo_total) AS ingresos_totales,
    SUM(fsp.costo_total) / da.años_experiencia AS ingresos_por_año_experiencia
FROM
    Fact_Servicios_Prestados fsp
JOIN
    Dim_Asesor da ON fsp.asesor_id = da.asesor_id
GROUP BY
    da.nombre, da.años_experiencia
ORDER BY
    ingresos_totales DESC;



     '''

In [102]:
df7 = pd.read_sql(query, con=engine)
df7

,asesor,años_experiencia,ingresos_totales,ingresos_por_año_experiencia
0,Javier Ortega,7,29649.46,4235.637143
1,David Ruiz,12,25433.56,2119.463333
2,Laura Sánchez,8,20938.37,2617.296250
3,Sofia Díaz,5,19023.86,3804.772000
4,Pedro Martínez,10,12308.47,1230.847000
